In [1]:
import numpy as np
import pandas as pd
import datetime as dt

import plotly
import plotly.express as px
import plotly.graph_objects as go

import chart_studio.tools as tls

In [2]:
# get the path to our data
path = r"F:\GitHub\Projects\MyLifeinData\data"
# read in our data
smt = pd.read_csv(path + r"\timeslots.csv")

In [3]:
# convert our string object into a datetime object
smt["Day"] = [dt.datetime.strptime(str(date), "%Y%m%d") for date in smt["Day"]]

In [48]:
# get the YYYY, MM, and DD columns from our datetime object
years = pd.DataFrame([date.year for date in (smt["Day"])], columns=["Year"])
months = pd.DataFrame([date.month for date in (smt["Day"])], columns=["Month"])
days = pd.DataFrame([date.day for date in (smt["Day"])], columns=["Day"])

# get the month abbreviation
months_ab = pd.DataFrame([date.strftime('%b') for date in (smt["Day"])], columns=["Month_ab"])

# keep only the columns that we really need
grouped_df = pd.concat([smt["Activity Category"], smt["Duration ms"]/360000, years, months, days, months_ab], axis=1)

# sum each of our groups to prep our data for plotting
grouped_df = grouped_df.groupby(["Year", "Month", "Month_ab", "Activity Category"]).sum().reset_index()
grouped_df = grouped_df.rename(columns={"Duration ms" : "Duration (Hours)"})

# split our data to not be past May, 2019
upto_may = [i for i, x in enumerate(grouped_df["Month"]) if x < 5]
grouped_df = grouped_df.iloc[upto_may]

# remove unnecessary categories
less_groups = [i for i, x in enumerate(grouped_df["Activity Category"]) if x not in ["Chores", "Health & Hygiene",
                                                                                "Resting", "Social", "Transport",
                                                                                "Sport & Fitness"]]
grouped_df = grouped_df.iloc[less_groups]

# remove unnecessary columns
grouped_df = grouped_df.drop(["Day", "Month"], axis=1)


# get our x-axis correct
grouped_df["Year-Month"] = grouped_df["Year"].map(str) + "-" + grouped_df["Month_ab"].map(str)

In [49]:
grouped_df

,Year,Month_ab,Activity Category,Duration (Hours),Year-Month
4,2019,Jan,Art & Entertainment,1617.094022,2019-Jan
7,2019,Jan,Personal Life,298.168964,2019-Jan
8,2019,Jan,Professional Life,1502.552828,2019-Jan
10,2019,Jan,Sleep,2495.570658,2019-Jan
13,2019,Feb,Art & Entertainment,1853.464050,2019-Feb
16,2019,Feb,Personal Life,105.120622,2019-Feb
17,2019,Feb,Professional Life,1526.516703,2019-Feb
19,2019,Feb,Sleep,2212.877956,2019-Feb
22,2019,Mar,Art & Entertainment,2197.282911,2019-Mar
25,2019,Mar,Personal Life,198.251761,2019-Mar


,Year,Month_ab,Activity Category,Duration (Hours),Year-Month
7,2019,Jan,Personal Life,298.168964,2019-Jan
16,2019,Feb,Personal Life,105.120622,2019-Feb
25,2019,Mar,Personal Life,198.251761,2019-Mar
35,2019,Apr,Personal Life,410.041203,2019-Apr


In [60]:
# initialize our empty figure
fig = go.Figure()

# for each of our categorys, we want to create a df that only has that category in it
for category in np.unique(grouped_df["Activity Category"]):
    
    split_df = grouped_df[grouped_df["Activity Category"] == category]
    
    # add a trace based on the data in our new 'split_df'
    fig.add_trace(go.Bar(
        x=split_df["Month_ab"],
        y=split_df["Duration (Hours)"],
        name=category
    ))

# modify the mode and tickangle of the xaxis, resulting in grouped bars with rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=-45)


# finally, update the title, y-axis label, and legend location
fig.update_layout(
    title="Monthly Time Spent",
    yaxis=dict(title="Hours in Each Category"),
    legend=dict(x=0, y=1.2)
)

fig.show()

In [40]:
fig = px.bar(grouped_df, x="Month_ab", y="Duration (Hours)", color='Activity Category', barmode='group')



fig.show()

In [7]:
need = '<script src="https://cdn.plot.ly/plotly-latest.min.js"></script> '
out = plotly.offline.plot(fig, include_plotlyjs=False, output_type='div')
print(need + out)

<script src="https://cdn.plot.ly/plotly-latest.min.js"></script> <div>
        
        
            <div id="44d5db49-d20d-4694-9b89-331a5f7041b1" class="plotly-graph-div" style="height:600px; width:100%;"></div>
            <script type="text/javascript">
                
                    window.PLOTLYENV=window.PLOTLYENV || {};
                    
                if (document.getElementById("44d5db49-d20d-4694-9b89-331a5f7041b1")) {
                    Plotly.newPlot(
                        '44d5db49-d20d-4694-9b89-331a5f7041b1',
                        [{"alignmentgroup": "True", "hoverlabel": {"namelength": 0}, "hovertemplate": "Activity Category=Art & Entertainment<br>Month=%{x}<br>Duration (Hours)=%{y}", "legendgroup": "Activity Category=Art & Entertainment", "marker": {"color": "#636efa"}, "name": "Activity Category=Art & Entertainment", "offsetgroup": "Activity Category=Art & Entertainment", "orientation": "v", "showlegend": true, "textposition": "auto", "type": "bar", "x